In [183]:
import kineticstoolkit.lab as ktk
import pandas as pd
# Set an interactive backend, not required if already enabled in Spyder
%matplotlib qt5
import numpy as np
import matplotlib.pyplot as plt
import os

In [184]:
# Relative path to the CSV file
Subject01_SRdata_path = r"../SRrawdata/Subject01/111_f_2024-11-04_13h07.58.818.csv"

# Read the CSV file into a DataFrame
Subject01_SRdata = pd.read_csv(Subject01_SRdata_path)
# Read the C3D fiel into a Dictionary
Subject01_MCdata_path = r"../MCrawdata/Subject01/Ex_V2_S0001.c3d"

# Read the C3D fiel into a Dictionary
Subject01_MCdata = ktk.read_c3d(Subject01_MCdata_path)['Points']

In [185]:
# # Relative path to the CSV file
# Subject01_SRdata_path = r"../SRrawdata/Subject02/222_f_2024-11-09_17h21.29.442.csv"

# # Read the CSV file into a DataFrame
# Subject01_SRdata = pd.read_csv(Subject01_SRdata_path)
# # Read the C3D file into a Dictionary
# Subject01_MCdata_path = r"../MCrawdata/Subject02/Ex_V2_S0002.c3d"

# # Read the C3D fiel into a Dictionary
# Subject01_MCdata = ktk.read_c3d(Subject01_MCdata_path)['Points']

In [186]:
# # Relative path to the CSV file
# Subject01_SRdata_path = r"../SRrawdata/Subject03/333_f_2024-11-09_19h07.57.263.csv"

# # Read the CSV file into a DataFrame
# Subject01_SRdata = pd.read_csv(Subject01_SRdata_path)
# # Read the C3D file into a Dictionary
# Subject01_MCdata_path = r"../MCrawdata/Subject03/Ex_V2_S0003.c3d"

# # Read the C3D fiel into a Dictionary
# Subject01_MCdata = ktk.read_c3d(Subject01_MCdata_path)['Points']

In [187]:
interconnections = dict()

interconnections["Hand"] = {
    "Color": (0, 0.5, 1),  # In (R,G,B) format (here, greenish blue)
    "Links": [  # List of lines that span lists of markers
        ["R_L_H", "R_M_H", "R_M_W", "R_L_W"],
        ['R_L_H','R_L_W']
    ],
}

interconnections["Thorax"] = {
    "Color": (0.5, 1, 0.5),
    "Links": [
        ["R_S", "L_S","Back"],
        ['R_S','Back']
    ],
}

interconnections["Belly"] = {
    "Color": (0.5, 1, 0.5),
    "Links": [
        ["R_H", "L_H","Back"],
        ['R_H','Back']
    ],
}


interconnections["Arm"] = {
    "Color": (1, 0.5, 1),
    "Links": [
        ['R_L_E', "R_L_W","R_M_W",'R_M_E'],
        ['R_L_E',"R_S",'R_M_E'],
        ['R_L_E','R_M_E'],
    ],
}



In [188]:
# Assuming SRdata is your DataFrame
# Initialize variables to store trial count and start/end times
trial_count = 0
start_end_times = []

# Iterate through each row in the 'moving_mouse.time' column
for i, row in Subject01_SRdata['moving_mouse.time'].items():
    if pd.notna(row):
        # Increment trial count
        trial_count += 1
        
        # Convert the row to a list and get the first and last element
        time_series = eval(row)  # Using eval to convert string representation of lists to actual lists
        start_time = time_series[0]
        end_time = time_series[-1]
        
        # Store the start and end times
        start_end_times.append((start_time, end_time))

In [190]:

Subject01_MCdata = Subject01_MCdata.ui_edit_events()

In [191]:
index_expstart = Subject01_MCdata.get_index_at_event('Expstart')

In [192]:
# First start time
first_start_time = start_end_times[0][0]-Subject01_MCdata.time[index_expstart]

# Sync all times by subtracting the first start time
synced_start_end_times = [(start - first_start_time, end - first_start_time) for start, end in start_end_times]

In [193]:
# Set up the plot
plt.figure(figsize=(10, 2))

# Plot each event as a line on the timeline
for (start, end) in synced_start_end_times:
    plt.plot([start, end], [0, 0], marker='|', color='b', linewidth=1)

# Label the axes
plt.xlabel("Time (s)")
plt.yticks([])  # Hide y-axis labels since we don't need them
plt.title("Event Timeline")

# Add grid for better visibility
plt.grid(True, which='both', axis='x', linestyle='--', linewidth=0.5)

Subject01_MCdata.plot(['PEN'])

# Show the plot
plt.show()

## Create Local corrdinate system

### Arm coordinate system

In [194]:
def get_frames(markers):
    frames = ktk.TimeSeries(time=markers.time)

    # ## Global
    # The global coordinate system was recorded with:
    #            z up, y forward and x to the right.
    # The joint coordinates are in y up, x forward, and z to the right.
    # Let's create a frame that captures this.
    global_transform = ktk.geometry.create_transforms(
        seq="xy", angles=[[-90, -90]], degrees=True
    )

    # ## pelvis
    pelvis_origin = (markers.data["R_H"] + markers.data["L_H"]) / 2
    pelvis_z = markers.data["L_H"] - markers.data["R_H"]
    pelvis_yz = markers.data["Back"] - pelvis_origin

    frames.data["Pelvis"] = ktk.geometry.create_frames(
        origin=pelvis_origin, z=pelvis_z, yz=pelvis_yz
    )

    # ## Thorax
    thorax_origin = (markers.data["L_S"] + markers.data["R_S"]) / 2
    thorax_z = markers.data["L_S"] - markers.data["R_S"]
    thorax_yz = thorax_origin - markers.data["Back"]

    frames.data["Thorax"] = ktk.geometry.create_frames(
        origin=thorax_origin, z=thorax_z, yz=thorax_yz
    )

    # ## Upper arms
    # ### Right upper arm
    r_arm_origin = markers.data["R_S"]
    r_arm_y = markers.data["R_S"] -0.5*(markers.data["R_L_E"] + markers.data["R_M_E"]) 
    r_arm_yz = markers.data["R_L_E"] - markers.data["R_M_E"]

    frames.data["R_Arm"] = ktk.geometry.create_frames(
        origin=r_arm_origin, y=r_arm_y, yz=r_arm_yz
    )


    # ## Forearms
    # ### Right forearm
    r_forearm_origin = markers.data["R_M_W"]
    r_forearm_y = 0.5*(markers.data["R_L_E"] + markers.data["R_M_E"])  - 0.5*(markers.data["R_L_W"]+markers.data["R_M_W"])
    r_forearm_yz = markers.data["R_M_W"] - markers.data["R_L_W"]

    frames.data["R_Forearm"] = ktk.geometry.create_frames(
        origin=r_forearm_origin, y=r_forearm_y, yz=r_forearm_yz
    )

    # ## Hands
    # ### Right hand
    r_hand_origin = markers.data["R_L_W"]
    r_hand_z = markers.data["R_L_W"] - markers.data["R_M_W"]
    r_hand_yz = (markers.data["R_M_H"] + markers.data["R_L_H"]) / 2 - 0.5*(markers.data["R_L_W"]+markers.data["R_M_W"])
    

    frames.data["R_Hand"] = ktk.geometry.create_frames(
        origin=r_hand_origin, z=r_hand_z, yz=r_hand_yz
    )

    return frames, global_transform

In [195]:
framesV2,global_transformV2 = get_frames(Subject01_MCdata)

RuntimeWarning [c:\Users\86153\anaconda3\envs\ktk\Lib\site-packages\kineticstoolkit\geometry.py:592] invalid value encountered in divide


In [196]:
def get_homogeneous_angle(frames, global_transform):
    homogeneous = ktk.TimeSeries(time=frames.time)

    if len(frames.time) == 0:
        nan_array = np.full((0, 4, 4), np.nan)
        for key in [
            "pelvis",
            "pelvis_to_thorax",
            "thorax_to_r_arm",
            "R_arm_to_forearm",
            "R_forearm_to_hand",
        ]:
            homogeneous.data[key] = nan_array
        return homogeneous
    try:
        homogeneous.data["pelvis"] = ktk.geometry.get_local_coordinates(
            frames.data["Pelvis"], global_transform
        )
        homogeneous.data["pelvis_to_thorax"] = ktk.geometry.get_local_coordinates(
            frames.data["Thorax"], frames.data["Pelvis"]
        )
        homogeneous.data["thorax_to_r_arm"] = ktk.geometry.get_local_coordinates(
            frames.data["R_Arm"], frames.data["Thorax"]
        )
        homogeneous.data["R_arm_to_forearm"] = ktk.geometry.get_local_coordinates(
            frames.data["R_Forearm"], frames.data["R_Arm"]
        )
        homogeneous.data["R_forearm_to_hand"] = ktk.geometry.get_local_coordinates(
            frames.data["R_Hand"], frames.data["R_Forearm"]
        )

    except Exception as e:
        print(f"An error occurred: {e}")
        
    return homogeneous

In [197]:
get_homogeneous_angle(framesV2,global_transformV2)

TimeSeries with attributes:
         time: <array of shape (310562,)>
         data: <dict with 5 entries>
    time_info: {'Unit': 's'}
    data_info: {}
       events: []

In [198]:

homogeneous = ktk.TimeSeries(time=Subject01_MCdata.time)

global_transform = ktk.geometry.create_transforms(
        seq="xy", angles=[[-90, -90]], degrees=True
    )
homogeneous.data["pelvis"] = ktk.geometry.get_local_coordinates(
    framesV2.data["Pelvis"], global_transform
)
homogeneous.data["pelvis_to_thorax"] = ktk.geometry.get_local_coordinates(
    framesV2.data["Thorax"], framesV2.data["Pelvis"]
        )
homogeneous.data["thorax_to_r_arm"] = ktk.geometry.get_local_coordinates(
    framesV2.data["R_Arm"], framesV2.data["Thorax"]
        )
homogeneous.data["R_arm_to_forearm"] = ktk.geometry.get_local_coordinates(
    framesV2.data["R_Forearm"], framesV2.data["R_Arm"]
        )
homogeneous.data["R_forearm_to_hand"] = ktk.geometry.get_local_coordinates(
            framesV2.data["R_Hand"], framesV2.data["R_Forearm"]
        )

In [199]:
euler = ktk.TimeSeries(time=framesV2.time)
euler.data["Pelvis"] = ktk.geometry.get_angles(
            homogeneous.data["pelvis"], "ZYZ", degrees=True
        )
euler.data["Thorax"] = ktk.geometry.get_angles(
            homogeneous.data["pelvis_to_thorax"], "YZX", degrees=True
        )
euler.data["R_Arm"] = ktk.geometry.get_angles(
            homogeneous.data["thorax_to_r_arm"], "YZX", degrees=True
        )
euler.data["R_Forearm"] = ktk.geometry.get_angles(
            homogeneous.data["R_arm_to_forearm"], "ZXY", degrees=True
        )
euler.data["R_Hand"] = ktk.geometry.get_angles(
            homogeneous.data["R_forearm_to_hand"], "XZY", degrees=True
        )


In [200]:
# plt.figure(figsize=(10, 2))
# # plt.plot(euler.time, euler.data["R_Arm"],label = "R_Arm" )
# # plt.plot(euler.time, euler.data["Thorax"],label = "Thorax" )
# # plt.plot(euler.time, euler.data["Pelvis"],label = "Pelvis")
# plt.plot(euler.time, euler.data["R_Forearm"],label = "R_Forearm")
# plt.title('Example Angle')

# plt.ylabel('deg')
# plt.legend()
# # plt.ylim(-100,100)
# plt.show()

In [201]:
euler_vels = ktk.filters.deriv(euler)
euler_vels.resample(euler.time, kind="cubic", in_place=True)

TimeSeries with attributes:
         time: <array of shape (310562,)>
         data: <dict with 5 entries>
    time_info: {'Unit': 's'}
    data_info: {}
       events: []

In [202]:
len(synced_start_end_times)

320

In [203]:
# Calculate the duration of each trial
durations = [end - start for start, end in synced_start_end_times]

# Calculate the mean and standard deviation of the durations
mean_duration = np.mean(durations)
std_duration = np.std(durations)

min_duration = np.min(durations)
max_duration = np.max(durations)

mean_duration, std_duration, min_duration, max_duration

(np.float64(0.8847459684402572),
 np.float64(0.6997254914168594),
 np.float64(0.4304182000923902),
 np.float64(9.977568000089377))

In [204]:
# Find closest indices in `euler_vels.time` for each trial's start and end time
trial_indices = [
    (
        np.abs(euler_vels.time - start).argmin(),  # Closest index to start time
        np.abs(euler_vels.time - end).argmin()     # Closest index to end time
    )
    for start, end in synced_start_end_times
]

# Use these indices to slice `euler_vels.time` for each trial
trial_segments = [euler_vels.time[start_idx:end_idx+1] for start_idx, end_idx in trial_indices]

# Check the first few segments as a sample
len(trial_segments)

320

### Forearm [0]

In [205]:
# Initialize list to store data for each trial's points
Angvel_R_Forearm0 = []

# Loop through each trial and extract point 1 and point 2 values
for i, (start_idx, end_idx) in enumerate(trial_indices):
    # Point 1: 21 indices after start_idx
    point1_idx = start_idx + 21 if start_idx + 21 <= end_idx else end_idx
    point1_time = euler_vels.time[point1_idx]
    point1_value = euler_vels.data["R_Forearm"][point1_idx, 0]
    point1_euler = euler.data["R_Forearm"][point1_idx, 0]

    
    # Point 2: Maximum value in the trial
    trial_data = euler_vels.data["R_Forearm"][start_idx:end_idx + 1, 0]
    trial_euler = euler.data["R_Forearm"][start_idx:end_idx + 1, 0]
    trial_time = euler_vels.time[start_idx:end_idx + 1]
    max_idx_in_trial = np.argmax(np.abs(trial_data))
    max_time = trial_time[max_idx_in_trial]
    max_value = trial_data[max_idx_in_trial]
    max_value_euler = trial_euler[max_idx_in_trial]
    
    # Add data for point 1
    Angvel_R_Forearm0.append({
        "Angle Name": "R_Forearm[0] move onset",
        "Trial ID": i + 1,
        "Time": point1_time,
        "Value": point1_value,
        "Euler": point1_euler
    })
    
    # Add data for point 2
    Angvel_R_Forearm0.append({
        "Angle Name": "R_Forearm[0] max vel",
        "Trial ID": i + 1,
        "Time": max_time,
        "Value": max_value,
        "Euler": max_value_euler
    })

# Convert the list to a DataFrame
Angvel_R_Forearm0 = pd.DataFrame(Angvel_R_Forearm0)


In [206]:
plt.figure(figsize=(10, 2))
# plt.plot(euler.time, euler.data["R_Arm"],label = "R_Arm" )
# plt.plot(euler.time, euler.data["Thorax"],label = "Thorax" )
# plt.plot(euler.time, euler.data["Pelvis"],label = "Pelvis")
plt.plot(euler.time, np.nan_to_num(euler.data["R_Forearm"]), label="R_Forearm")
plt.title('Example Angle')

plt.ylabel('deg')
plt.legend()
# plt.ylim(-100,100)
plt.show()

In [207]:
# plt.figure(figsize=(12, 6))

# for i, (start_idx, end_idx) in enumerate(trial_indices):
#     # Get the absolute data segment for the trial
#     trial_data = euler_vels.data["R_Forearm"][start_idx:end_idx + 1, 0]   # Using abs() on the first angle
#     trial_time = euler_vels.time[start_idx:end_idx + 1]

#     # Set color based on trial range
#     if i < 10:
#         color = 'grey'  # Familiarization: trials 1–10
#         label = 'Familiarization' if i == 0 else ""
#     elif 310 <= i < 321:
#         color = 'black'  # Baseline: trials 11–40
#         label = 'Baseline' if i == 10 else ""
#     else:
#         color = 'red'  # Rotation: trials 41–85
#         label = 'Rotation' if i == 40 else ""

#         # Point 1: 60 indices after start_idx 400ms 0.4s
#     point1_idx = start_idx + 30 if start_idx + 30 <= end_idx else end_idx
#     point1_time = euler_vels.time[point1_idx]
#     point1_value = euler_vels.data["R_Forearm"][point1_idx, 0]

#     # Point 2: Maximum value in the trial
#     max_idx_in_trial = np.argmax(np.abs(trial_data))  # Index relative to trial_data
#     max_time = trial_time[max_idx_in_trial]
#     max_value = trial_data[max_idx_in_trial]

#         # Plot the trial data segment for context
#     plt.plot(trial_time, trial_data,  color=color, label=label, alpha=0.5)

#     # Plot Point 1
#     plt.plot(point1_time, point1_value, 'o', color='blue', label='Move on set 200ms' if i == 0 else "")

#     # Plot Point 2 (maximum value)
#     plt.plot(max_time, max_value, 's', color='red', label='Max Value' if i == 0 else "")

# # Add labels and legend
# plt.ylabel('deg/s (absolute)')
# plt.title('Absolute R_Forearm Angle Across Phases')
# plt.legend(loc='upper right')
# plt.show()

### Forearm [2]

In [208]:
# Initialize list to store data for each trial's points
Angvel_R_Forearm2 = []

# Loop through each trial and extract point 1 and point 2 values
for i, (start_idx, end_idx) in enumerate(trial_indices):
    # Point 1: 21 indices after start_idx
    point1_idx = start_idx + 21 if start_idx + 21 <= end_idx else end_idx
    point1_time = euler_vels.time[point1_idx]
    point1_value = euler_vels.data["R_Forearm"][point1_idx, 2]
    point1_euler = euler.data["R_Forearm"][point1_idx, 2]
    
    # Point 2: Maximum value in the trial
    trial_data = euler_vels.data["R_Forearm"][start_idx:end_idx + 1, 2]
    trial_euler = euler.data["R_Forearm"][start_idx:end_idx + 1, 2]
    trial_time = euler_vels.time[start_idx:end_idx + 1]
    max_idx_in_trial = np.argmax(np.abs(trial_data))
    max_time = trial_time[max_idx_in_trial]
    max_value = trial_data[max_idx_in_trial]
    max_value_euler = trial_euler[max_idx_in_trial]
    
    # Add data for point 1
    Angvel_R_Forearm2.append({
        "Angle Name": "R_Forearm[2] move onset",
        "Trial ID": i + 1,
        "Time": point1_time,
        "Value": point1_value,
        "Euler": point1_euler
    })
    
    # Add data for point 2
    Angvel_R_Forearm2.append({
        "Angle Name": "R_Forearm[2] max vel",
        "Trial ID": i + 1,
        "Time": max_time,
        "Value": max_value,
        "Euler": max_value_euler
    })

# Convert the list to a DataFrame
Angvel_R_Forearm2 = pd.DataFrame(Angvel_R_Forearm2)


### Arm[1]

In [209]:
# Initialize list to store data for each trial's points
Angvel_R_Arm1 = []

# Loop through each trial and extract point 1 and point 2 values
for i, (start_idx, end_idx) in enumerate(trial_indices):
    # Point 1: 21 indices after start_idx
    point1_idx = start_idx + 21 if start_idx + 21 <= end_idx else end_idx
    point1_time = euler_vels.time[point1_idx]
    point1_value = euler_vels.data["R_Arm"][point1_idx, 1]
    point1_euler = euler.data["R_Arm"][point1_idx, 1]
    
    # Point 2: Maximum value in the trial
    trial_data = euler_vels.data["R_Arm"][start_idx:end_idx + 1, 1]
    trial_euler = euler.data["R_Arm"][start_idx:end_idx + 1, 1]
    trial_time = euler_vels.time[start_idx:end_idx + 1]
    max_idx_in_trial = np.argmax(np.abs(trial_data))
    max_time = trial_time[max_idx_in_trial]
    max_value = trial_data[max_idx_in_trial]
    max_value_euler = trial_euler[max_idx_in_trial]
    
    # Add data for point 1
    Angvel_R_Arm1.append({
        "Angle Name": "R_Arm[1] move onset",
        "Trial ID": i + 1,
        "Time": point1_time,
        "Value": point1_value,
        "Euler": point1_euler
    })
    
    # Add data for point 2
    Angvel_R_Arm1.append({
        "Angle Name": "R_Arm[1] max vel",
        "Trial ID": i + 1,
        "Time": max_time,
        "Value": max_value,
        "Euler": max_value_euler
    })

# Convert the list to a DataFrame
Angvel_R_Arm1 = pd.DataFrame(Angvel_R_Arm1)


### Arm[2]

In [210]:
# Initialize list to store data for each trial's points
Angvel_R_Arm2 = []

# Loop through each trial and extract point 1 and point 2 values
for i, (start_idx, end_idx) in enumerate(trial_indices):
    # Point 1: 21 indices after start_idx
    point1_idx = start_idx + 21 if start_idx + 21 <= end_idx else end_idx
    point1_time = euler_vels.time[point1_idx]
    point1_value = euler_vels.data["R_Arm"][point1_idx, 2]
    point1_euler = euler.data["R_Arm"][point1_idx, 2]
    
    # Point 2: Maximum value in the trial
    trial_data = euler_vels.data["R_Arm"][start_idx:end_idx + 1,2]
    trial_euler = euler.data["R_Arm"][start_idx:end_idx + 1, 2]
    trial_time = euler_vels.time[start_idx:end_idx + 1]
    max_idx_in_trial = np.argmax(np.abs(trial_data))
    max_time = trial_time[max_idx_in_trial]
    max_value = trial_data[max_idx_in_trial]
    max_value_euler = trial_euler[max_idx_in_trial]
    
    # Add data for point 1
    Angvel_R_Arm2.append({
        "Angle Name": "R_Arm[2] move onset",
        "Trial ID": i + 1,
        "Time": point1_time,
        "Value": point1_value,
        "Euler": point1_euler
    })
    
    # Add data for point 2
    Angvel_R_Arm2.append({
        "Angle Name": "R_Arm[2] max vel",
        "Trial ID": i + 1,
        "Time": max_time,
        "Value": max_value,
        "Euler": max_value_euler
    })

# Convert the list to a DataFrame
Angvel_R_Arm2 = pd.DataFrame(Angvel_R_Arm2)


### Hand[1]

In [211]:
# Initialize list to store data for each trial's points
Angvel_R_Hand1 = []

# Loop through each trial and extract point 1 and point 2 values
for i, (start_idx, end_idx) in enumerate(trial_indices):
    # Point 1: 21 indices after start_idx
    point1_idx = start_idx + 21 if start_idx + 21 <= end_idx else end_idx
    point1_time = euler_vels.time[point1_idx]
    point1_value = euler_vels.data["R_Hand"][point1_idx, 1]
    point1_euler = euler.data["R_Hand"][point1_idx, 1]
    
    # Point 2: Maximum value in the trial
    trial_data = euler_vels.data["R_Hand"][start_idx:end_idx + 1,1]
    trial_euler = euler.data["R_Hand"][start_idx:end_idx + 1, 1]
    trial_time = euler_vels.time[start_idx:end_idx + 1]
    max_idx_in_trial = np.argmax(np.abs(trial_data))
    max_time = trial_time[max_idx_in_trial]
    max_value = trial_data[max_idx_in_trial]
    max_value_euler = trial_euler[max_idx_in_trial]
    
    # Add data for point 1
    Angvel_R_Hand1.append({
        "Angle Name": "R_Hand[1] move onset",
        "Trial ID": i + 1,
        "Time": point1_time,
        "Value": point1_value,
        "Euler": point1_euler
    })
    
    # Add data for point 2
    Angvel_R_Hand1.append({
        "Angle Name": "R_Hand[1] max vel",
        "Trial ID": i + 1,
        "Time": max_time,
        "Value": max_value,
        "Euler": max_value_euler
    })

# Convert the list to a DataFrame
Angvel_R_Hand1 = pd.DataFrame(Angvel_R_Hand1)


### Throax

In [212]:
# Initialize list to store data for each trial's points
Angvel_throax0 = []

# Loop through each trial and extract point 1 and point 2 values
for i, (start_idx, end_idx) in enumerate(trial_indices):
    # Point 1: 21 indices after start_idx
    point1_idx = start_idx + 21 if start_idx + 21 <= end_idx else end_idx
    point1_time = euler_vels.time[point1_idx]
    point1_value = euler_vels.data["Thorax"][point1_idx, 0]
    point1_euler = euler.data["Thorax"][point1_idx, 0]
    
    # Point 2: Maximum value in the trial
    trial_data = euler_vels.data["Thorax"][start_idx:end_idx + 1,0]
    trial_euler = euler.data["Thorax"][start_idx:end_idx + 1, 0]
    trial_time = euler_vels.time[start_idx:end_idx + 1]
    max_idx_in_trial = np.argmax(np.abs(trial_data))
    max_time = trial_time[max_idx_in_trial]
    max_value = trial_data[max_idx_in_trial]
    max_value_euler = trial_euler[max_idx_in_trial]
    
    # Add data for point 1
    Angvel_throax0.append({
        "Angle Name": "Thorax[0] move onset",
        "Trial ID": i + 1,
        "Time": point1_time,
        "Value": point1_value,
        "Euler": point1_euler
    })
    
    # Add data for point 2
    Angvel_throax0.append({
        "Angle Name": "Thorax[0] max vel",
        "Trial ID": i + 1,
        "Time": max_time,
        "Value": max_value,
        "Euler": max_value_euler
    })

# Convert the list to a DataFrame
Angvel_throax0 = pd.DataFrame(Angvel_throax0)


In [213]:
# Initialize list to store data for each trial's points
Angvel_throax1 = []

# Loop through each trial and extract point 1 and point 2 values
for i, (start_idx, end_idx) in enumerate(trial_indices):
    # Point 1: 21 indices after start_idx
    point1_idx = start_idx + 21 if start_idx + 21 <= end_idx else end_idx
    point1_time = euler_vels.time[point1_idx]
    point1_value = euler_vels.data["Thorax"][point1_idx, 1]
    point1_euler = euler.data["Thorax"][point1_idx, 1]
    
    # Point 2: Maximum value in the trial
    trial_data = euler_vels.data["Thorax"][start_idx:end_idx + 1,1]
    trial_euler = euler.data["Thorax"][start_idx:end_idx + 1, 1]
    trial_time = euler_vels.time[start_idx:end_idx + 1]
    max_idx_in_trial = np.argmax(np.abs(trial_data))
    max_time = trial_time[max_idx_in_trial]
    max_value = trial_data[max_idx_in_trial]
    max_value_euler = trial_euler[max_idx_in_trial]
    
    
    # Add data for point 1
    Angvel_throax1.append({
        "Angle Name": "Thorax[1] move onset",
        "Trial ID": i + 1,
        "Time": point1_time,
        "Value": point1_value,
        "Euler": point1_euler
    })
    
    # Add data for point 2
    Angvel_throax1.append({
        "Angle Name": "Thorax[1] max vel",
        "Trial ID": i + 1,
        "Time": max_time,
        "Value": max_value,
        "Euler": max_value_euler
    })

# Convert the list to a DataFrame
Angvel_throax1 = pd.DataFrame(Angvel_throax1)


In [214]:
# Initialize list to store data for each trial's points
Angvel_throax2 = []

# Loop through each trial and extract point 1 and point 2 values
for i, (start_idx, end_idx) in enumerate(trial_indices):
    # Point 1: 21 indices after start_idx
    point1_idx = start_idx + 21 if start_idx + 21 <= end_idx else end_idx
    point1_time = euler_vels.time[point1_idx]
    point1_value = euler_vels.data["Thorax"][point1_idx, 2]
    point1_euler = euler.data["Thorax"][point1_idx, 2]
    
    # Point 2: Maximum value in the trial
    trial_data = euler_vels.data["Thorax"][start_idx:end_idx + 1,2]
    trial_euler = euler.data["Thorax"][start_idx:end_idx + 1, 2]
    trial_time = euler_vels.time[start_idx:end_idx + 1]
    max_idx_in_trial = np.argmax(np.abs(trial_data))
    max_time = trial_time[max_idx_in_trial]
    max_value = trial_data[max_idx_in_trial]
    max_value_euler = trial_euler[max_idx_in_trial]
    
    
    # Add data for point 1
    Angvel_throax2.append({
        "Angle Name": "Thorax[2] move onset",
        "Trial ID": i + 1,
        "Time": point1_time,
        "Value": point1_value,
        "Euler": point1_euler
    })
    
    # Add data for point 2
    Angvel_throax2.append({
        "Angle Name": "Thorax[2] max vel",
        "Trial ID": i + 1,
        "Time": max_time,
        "Value": max_value,
        "Euler": max_value_euler
    })

# Convert the list to a DataFrame
Angvel_throax2 = pd.DataFrame(Angvel_throax2)


### Pelvis

In [215]:
# Initialize list to store data for each trial's points
Angvel_pelvis0 = []

# Loop through each trial and extract point 1 and point 2 values
for i, (start_idx, end_idx) in enumerate(trial_indices):
    # Point 1: 21 indices after start_idx
    point1_idx = start_idx + 21 if start_idx + 21 <= end_idx else end_idx
    point1_time = euler_vels.time[point1_idx]
    point1_value = euler_vels.data["Pelvis"][point1_idx, 0]
    point1_euler = euler.data["Pelvis"][point1_idx, 0]
    
    # Point 2: Maximum value in the trial
    trial_data = euler_vels.data["Pelvis"][start_idx:end_idx + 1,0]
    trial_euler = euler.data["Pelvis"][start_idx:end_idx + 1, 0]
    trial_time = euler_vels.time[start_idx:end_idx + 1]
    max_idx_in_trial = np.argmax(np.abs(trial_data))
    max_time = trial_time[max_idx_in_trial]
    max_value = trial_data[max_idx_in_trial]
    max_value_euler = trial_euler[max_idx_in_trial]
    
    
    # Add data for point 1
    Angvel_pelvis0.append({
        "Angle Name": "Pelvis[0] move onset",
        "Trial ID": i + 1,
        "Time": point1_time,
        "Value": point1_value,
        "Euler": point1_euler
    })
    
    # Add data for point 2
    Angvel_pelvis0.append({
        "Angle Name": "Pelvis[0] max vel",
        "Trial ID": i + 1,
        "Time": max_time,
        "Value": max_value,
        "Euler": max_value_euler
    })

# Convert the list to a DataFrame
Angvel_pelvis0 = pd.DataFrame(Angvel_pelvis0)


In [216]:
# Initialize list to store data for each trial's points
Angvel_pelvis1 = []

# Loop through each trial and extract point 1 and point 2 values
for i, (start_idx, end_idx) in enumerate(trial_indices):
    # Point 1: 21 indices after start_idx
    point1_idx = start_idx + 21 if start_idx + 21 <= end_idx else end_idx
    point1_time = euler_vels.time[point1_idx]
    point1_value = euler_vels.data["Pelvis"][point1_idx, 1]
    point1_euler = euler.data["Pelvis"][point1_idx, 1]
    
    # Point 2: Maximum value in the trial
    trial_data = euler_vels.data["Pelvis"][start_idx:end_idx + 1,1]
    trial_euler = euler.data["Pelvis"][start_idx:end_idx + 1,1]
    trial_time = euler_vels.time[start_idx:end_idx + 1]
    max_idx_in_trial = np.argmax(np.abs(trial_data))
    max_time = trial_time[max_idx_in_trial]
    max_value = trial_data[max_idx_in_trial]
    max_value_euler = trial_euler[max_idx_in_trial]
    # Add data for point 1
    Angvel_pelvis1.append({
        "Angle Name": "Pelvis[1] move onset",
        "Trial ID": i + 1,
        "Time": point1_time,
        "Value": point1_value,
        "Euler": point1_euler
    })
    
    # Add data for point 2
    Angvel_pelvis1.append({
        "Angle Name": "Pelvis[1] max vel",
        "Trial ID": i + 1,
        "Time": max_time,
        "Value": max_value,
        "Euler": max_value_euler
    })

# Convert the list to a DataFrame
Angvel_pelvis1 = pd.DataFrame(Angvel_pelvis1)


In [217]:
# Initialize list to store data for each trial's points
Angvel_pelvis2 = []

# Loop through each trial and extract point 1 and point 2 values
for i, (start_idx, end_idx) in enumerate(trial_indices):
    # Point 1: 21 indices after start_idx
    point1_idx = start_idx + 21 if start_idx + 21 <= end_idx else end_idx
    point1_time = euler_vels.time[point1_idx]
    point1_value = euler_vels.data["Pelvis"][point1_idx, 2]
    point1_euler = euler.data["Pelvis"][point1_idx, 2]
    
    # Point 2: Maximum value in the trial
    trial_data = euler_vels.data["Pelvis"][start_idx:end_idx + 1,2]
    trial_euler = euler.data["Pelvis"][start_idx:end_idx + 1, 2]
    trial_time = euler_vels.time[start_idx:end_idx + 1]
    max_idx_in_trial = np.argmax(np.abs(trial_data))
    max_time = trial_time[max_idx_in_trial]
    max_value = trial_data[max_idx_in_trial]
    max_value_euler = trial_euler[max_idx_in_trial]
    
    # Add data for point 1
    Angvel_pelvis2.append({
        "Angle Name": "Pelvis[2] move onset",
        "Trial ID": i + 1,
        "Time": point1_time,
        "Value": point1_value,
        "Euler": point1_euler
    })
    
    # Add data for point 2
    Angvel_pelvis2.append({
        "Angle Name": "Pelvis[2] max vel",
        "Trial ID": i + 1,
        "Time": max_time,
        "Value": max_value,
        "Euler": max_value_euler
    })

# Convert the list to a DataFrame
Angvel_pelvis2 = pd.DataFrame(Angvel_pelvis2)


In [218]:
# List of DataFrames
all_joints_list = [
    Angvel_R_Forearm0, Angvel_R_Forearm2, Angvel_R_Arm1, Angvel_R_Arm2,
    Angvel_R_Hand1, Angvel_throax0, Angvel_throax1, Angvel_throax2,
    Angvel_pelvis0, Angvel_pelvis1, Angvel_pelvis2
]

# Concatenate all data frames
all_joints_df = pd.concat(all_joints_list, ignore_index=True)

# Ensure the data is sorted by 'Trial ID'
all_joints_df = all_joints_df.sort_values(by=["Trial ID", "Angle Name"]).reset_index(drop=True)



In [219]:
Rotation_data_early = all_joints_df[(all_joints_df['Trial ID'] >= 11) & (all_joints_df['Trial ID'] <= 110)]
Rotation_data_late = all_joints_df[(all_joints_df['Trial ID'] >= 211) & (all_joints_df['Trial ID'] <= 310)]

In [220]:
# Step 1: Create pivot tables
early_value_data = Rotation_data_early.pivot_table(index='Trial ID', columns='Angle Name', values='Value')
early_euler_data = Rotation_data_early.pivot_table(index='Trial ID', columns='Angle Name', values='Euler')

# Step 2: Combine the two DataFrames into one
early_data = pd.concat([early_value_data, early_euler_data], axis=1)

# Step 3: Rename the columns for clarity
# Create new column names for Value and Euler
value_columns = [f"{angle}_Value" for angle in early_value_data.columns]
euler_columns = [f"{angle}_Euler" for angle in early_euler_data.columns]

# Assign the new column names
early_data.columns = value_columns + euler_columns

In [221]:
early_cov_matrix = early_data.cov()
early_det_value = np.linalg.det(early_cov_matrix)

In [222]:
# Step 1: Create pivot tables
late_value_data = Rotation_data_late.pivot_table(index='Trial ID', columns='Angle Name', values='Value')
late_euler_data = Rotation_data_late.pivot_table(index='Trial ID', columns='Angle Name', values='Euler')

# Step 2: Combine the two DataFrames into one
late_data = pd.concat([late_value_data, late_euler_data], axis=1)

# Step 3: Rename the columns for clarity
# Create new column names for Value and Euler
value_columns = [f"{angle}_Value" for angle in late_value_data.columns]
euler_columns = [f"{angle}_Euler" for angle in late_euler_data.columns]

# Assign the new column names
late_data.columns = value_columns + euler_columns

In [223]:
late_cov_matrix = late_data.cov()
late_det_value = np.linalg.det(late_cov_matrix)

In [224]:
early_det_value

np.float64(9.312548802432506e+44)

In [225]:
late_det_value

np.float64(2.9516510658055335e+42)

# PyOPLS

In [226]:
# Assuming SRdata is your DataFrame
# Initialize variables to store trial count and start/end times
trial_count = 0
Endx = []
Endy = []

# Iterate through each row in the 'moving_mouse.time' column
for i, row in Subject01_SRdata['moving_mouse.x'].items():
    if pd.notna(row):
        # Increment trial count
        trial_count += 1
        
        # Convert the row to a list and get the first and last element
        time_series = eval(row)  # Using eval to convert string representation of lists to actual lists
        endM_x= time_series[-1]
        
        # Store the start and end times
        Endx.append((endM_x))

# Iterate through each row in the 'moving_mouse.time' column
for i, row in Subject01_SRdata['moving_mouse.y'].items():
    if pd.notna(row):
        # Increment trial count
        trial_count += 1
        
        # Convert the row to a list and get the first and last element
        time_series = eval(row)  # Using eval to convert string representation of lists to actual lists
        endM_y= time_series[-1]
        
        # Store the start and end times
        Endy.append((endM_y))

In [227]:
# Calculate the angle for each (x, y) pair
angles = np.arctan2(Endy, Endx)

# Optionally, convert from radians to degrees
angles_degrees = np.degrees(angles)

rotation_angles = angles_degrees-60

In [228]:
# Select the elements from index 11 to 110 for early_angerror
early_angerror = rotation_angles[10:110]

# Select the elements from index 210 to 310 for end_angerror
late_angerror = rotation_angles[210:310]

In [229]:
from sklearn.metrics import roc_curve, roc_auc_score
from pyopls import OPLS
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import cross_val_predict, LeaveOneOut
from sklearn.metrics import r2_score, accuracy_score

In [230]:
early_data.shape

(100, 44)

In [231]:
has_nan = early_data.isna().any().any()
print("Does early_data contain NaN values?", has_nan)
nan_rows = early_data[early_data.isna().any(axis=1)]
print("Rows containing NaN values:")
print(nan_rows)


Does early_data contain NaN values? False
Rows containing NaN values:
Empty DataFrame
Columns: [Pelvis[0] max vel_Value, Pelvis[0] move onset_Value, Pelvis[1] max vel_Value, Pelvis[1] move onset_Value, Pelvis[2] max vel_Value, Pelvis[2] move onset_Value, R_Arm[1] max vel_Value, R_Arm[1] move onset_Value, R_Arm[2] max vel_Value, R_Arm[2] move onset_Value, R_Forearm[0] max vel_Value, R_Forearm[0] move onset_Value, R_Forearm[2] max vel_Value, R_Forearm[2] move onset_Value, R_Hand[1] max vel_Value, R_Hand[1] move onset_Value, Thorax[0] max vel_Value, Thorax[0] move onset_Value, Thorax[1] max vel_Value, Thorax[1] move onset_Value, Thorax[2] max vel_Value, Thorax[2] move onset_Value, Pelvis[0] max vel_Euler, Pelvis[0] move onset_Euler, Pelvis[1] max vel_Euler, Pelvis[1] move onset_Euler, Pelvis[2] max vel_Euler, Pelvis[2] move onset_Euler, R_Arm[1] max vel_Euler, R_Arm[1] move onset_Euler, R_Arm[2] max vel_Euler, R_Arm[2] move onset_Euler, R_Forearm[0] max vel_Euler, R_Forearm[0] move onset_

In [232]:
late_angerror.shape

(100,)

In [233]:
# Forward-fill rows with NaNs based on the previous row
early_data = early_data.ffill(axis=0)

# Check if there are still any NaNs remaining in the first row
# If the first row has NaNs, they will remain as there is no previous row to copy from
early_data.iloc[0] = early_data.iloc[0].fillna(early_data.iloc[1])

##  Early

In [234]:
opls = OPLS(20)
self = opls.fit(early_data,early_angerror)
TP = self.T_ortho_ @ np.transpose(self.P_ortho_)
early_reduandant_cov_matrix = np.cov(TP)
early_reduandant_det_value = np.linalg.det(early_reduandant_cov_matrix)
Z = opls.transform(early_data)

early_related_cov_matrix = np.cov(Z)
early_related_det_value = np.linalg.det(early_related_cov_matrix)



In [235]:
# mean_duration = np.mean(early_reduandant_cov_matrix)
# mean_value = early_reduandant_cov_matrix.mean()
std_duration = np.std(early_reduandant_cov_matrix)

min_duration = np.min(early_reduandant_cov_matrix)
max_duration = np.max(early_reduandant_cov_matrix)


std_duration,min_duration,max_duration

(np.float64(0.27534324595724785),
 np.float64(-0.8516539711713791),
 np.float64(3.9136046014621333))

In [236]:
early_reduandant_det_value


np.float64(-0.0)

In [237]:
early_related_det_value

np.float64(0.0)

In [238]:
efficiency = early_reduandant_det_value / early_related_det_value


RuntimeWarning [C:\Users\86153\AppData\Local\Temp\ipykernel_17396\602075590.py:1] invalid value encountered in scalar divide


In [239]:
efficiency 

np.float64(nan)

In [240]:

from scipy.linalg import svd

def effective_pseudo_determinant(matrix, threshold=1e-10):
    """
    Calculate the pseudo-determinant of a matrix based on its effective rank.
    
    Parameters:
    matrix (np.ndarray): The input matrix.
    threshold (float): Threshold for considering singular values as non-zero.
    
    Returns:
    float: The pseudo-determinant based on effective rank.
    """
    # Compute singular values
    u, s, vh = svd(matrix)
    
    # Filter singular values by threshold to determine effective rank
    effective_singular_values = s[s > threshold]
    
    # Compute pseudo-determinant as the product of these singular values
    pseudo_det = np.prod(effective_singular_values)
    
    return pseudo_det

# Example usage:
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9],[0,2,0.2]])  # Example matrix (rank-deficient)
pseudo_det = effective_pseudo_determinant(A, threshold=1e-5)
print("Effective pseudo-determinant:", pseudo_det)

Effective pseudo-determinant: 27.92418306772827


In [241]:
pseudo_det = effective_pseudo_determinant(early_reduandant_cov_matrix, threshold=1e-1)

In [242]:
pseudo_det

np.float64(7181707.298423651)

In [243]:
pseudo_det_related = effective_pseudo_determinant(early_related_cov_matrix, threshold=1e-1)

In [244]:
pseudo_det_related

np.float64(0.0018842672125204162)

In [245]:
pseudo_efficiency  = pseudo_det/pseudo_det_related
pseudo_efficiency 


np.float64(3811405967.637319)

In [246]:
pseudo_early_cov_matrix = effective_pseudo_determinant(early_cov_matrix, threshold=1e-1)
pseudo_early_cov_matrix

np.float64(4.2509457677808296e+54)

In [247]:
pseudo_det*pseudo_det_related

np.float64(13532.25559233826)

## Late

In [248]:
has_nan = late_data.isna().any().any()
print("Does late_data contain NaN values?", has_nan)
nan_rows = late_data[late_data.isna().any(axis=1)]
print("Rows containing NaN values:")
print(nan_rows)


Does late_data contain NaN values? False
Rows containing NaN values:
Empty DataFrame
Columns: [Pelvis[0] max vel_Value, Pelvis[0] move onset_Value, Pelvis[1] max vel_Value, Pelvis[1] move onset_Value, Pelvis[2] max vel_Value, Pelvis[2] move onset_Value, R_Arm[1] max vel_Value, R_Arm[1] move onset_Value, R_Arm[2] max vel_Value, R_Arm[2] move onset_Value, R_Forearm[0] max vel_Value, R_Forearm[0] move onset_Value, R_Forearm[2] max vel_Value, R_Forearm[2] move onset_Value, R_Hand[1] max vel_Value, R_Hand[1] move onset_Value, Thorax[0] max vel_Value, Thorax[0] move onset_Value, Thorax[1] max vel_Value, Thorax[1] move onset_Value, Thorax[2] max vel_Value, Thorax[2] move onset_Value, Pelvis[0] max vel_Euler, Pelvis[0] move onset_Euler, Pelvis[1] max vel_Euler, Pelvis[1] move onset_Euler, Pelvis[2] max vel_Euler, Pelvis[2] move onset_Euler, R_Arm[1] max vel_Euler, R_Arm[1] move onset_Euler, R_Arm[2] max vel_Euler, R_Arm[2] move onset_Euler, R_Forearm[0] max vel_Euler, R_Forearm[0] move onset_E

In [249]:
# Forward-fill rows with NaNs based on the previous row
late_data = late_data.ffill(axis=0)

# Check if there are still any NaNs remaining in the first row
# If the first row has NaNs, they will remain as there is no previous row to copy from
late_data.iloc[0] = late_data.iloc[0].fillna(late_data.iloc[1])

In [250]:
opls = OPLS(20)

self = opls.fit(late_data,late_angerror)
TP = self.T_ortho_ @ np.transpose(self.P_ortho_)
late_reduandant_cov_matrix = np.cov(TP)
late_reduandant_det_value = np.linalg.det(early_reduandant_cov_matrix)
Z = opls.transform(late_data)

late_related_cov_matrix = np.cov(Z)
late_related_det_value = np.linalg.det(late_related_cov_matrix)



In [251]:
pseudo_det_late = effective_pseudo_determinant(late_reduandant_cov_matrix, threshold=1e-1)
pseudo_det_late

np.float64(7685128.803830537)

In [252]:
pseudo_det_related_late = effective_pseudo_determinant(late_related_cov_matrix, threshold=1e-1)
pseudo_det_related_late

np.float64(0.0026203593681432946)

In [253]:
pseudo_efficiency_late  = pseudo_det_late/pseudo_det_related_late
pseudo_efficiency_late 

np.float64(2932852988.510496)

In [254]:
pseudo_late_cov_matrix = effective_pseudo_determinant(late_cov_matrix, threshold=1e-1)
pseudo_late_cov_matrix

np.float64(2.583246169734779e+52)

In [255]:
pseudo_det_late*pseudo_det_related_late

np.float64(20137.79925650522)

# Movement complexity

In [256]:
from sklearn.decomposition import PCA

# Perform PCA
pca_early = PCA()
pca_early.fit(early_data)

# Calculate explained variance ratio for each component
early_explained_variance_ratio = pca_early.explained_variance_ratio_

# Determine number of PCs needed to explain 90% of variance
early_cumulative_variance = np.cumsum(early_explained_variance_ratio)
early_num_pcs_90 = np.argmax(early_cumulative_variance >= 0.90) + 1

# Count how many PCs explain more than 1% of variance
early_num_pcs_above_1_percent = np.sum(early_explained_variance_ratio > 0.01)

print(f"Number of PCs needed to explain 90% of variance: {early_num_pcs_90}")
print(f"Number of PCs that explain more than 1% of variance: {early_num_pcs_above_1_percent}")

Number of PCs needed to explain 90% of variance: 4
Number of PCs that explain more than 1% of variance: 7


In [257]:
# Perform PCA
pca_late = PCA()
pca_late.fit(late_data)

# Calculate explained variance ratio for each component
late_explained_variance_ratio = pca_late.explained_variance_ratio_

# Determine number of PCs needed to explain 90% of variance
late_cumulative_variance = np.cumsum(late_explained_variance_ratio)
late_num_pcs_90 = np.argmax(late_cumulative_variance >= 0.90) + 1

# Count how many PCs explain more than 1% of variance
late_num_pcs_above_1_percent = np.sum(late_explained_variance_ratio > 0.01)

print(f"Number of PCs needed to explain 90% of variance: {late_num_pcs_90}")
print(f"Number of PCs that explain more than 1% of variance: {late_num_pcs_above_1_percent}")

Number of PCs needed to explain 90% of variance: 3
Number of PCs that explain more than 1% of variance: 7


In [258]:
# Assuming early_angerror and late_angerror are arrays or lists with 100 elements each.
# Replace `x_values` with your actual x-coordinates if they are defined. If you just want indices as x-values:
x_values = range(100)

# Scatter plot for early_angerror
plt.scatter(x_values, early_angerror, label="Early Angle Error", color='blue')

# Scatter plot for late_angerror
plt.scatter(x_values, late_angerror, label="Late Angle Error", color='red')

# Adding labels and title
plt.xlabel("Index")
plt.ylabel("Angle Error")
plt.title("Scatter Plot of Early and Late Angle Errors")

# Show legend
plt.legend()

# Display the plot
plt.show()

In [259]:
print(pseudo_early_cov_matrix,pseudo_late_cov_matrix)

4.2509457677808296e+54 2.583246169734779e+52


In [260]:
print(pseudo_efficiency, pseudo_efficiency_late)

3811405967.637319 2932852988.510496


In [261]:
# print(f"The genearal variance in the early rotation : {early_det_value}")
# print(f"The genearal variance in the late rotation : {late_det_value}")

# print(f"The genearal variance in the early rotation using Pseudo-Determinant: {pseudo_early_cov_matrix}")
# print(f"The genearal variance in the late rotation using Pseudo-Determinant : {pseudo_late_cov_matrix}")

print(f"The Efficiency in the early rotation using Pseudo-Determinant : {pseudo_efficiency}")
print(f"The Efficiency in the late rotation using Pseudo-Determinant : {pseudo_efficiency_late}")

print(f"Number of PCs needed to explain 90% of variance - early: {early_num_pcs_90}")
print(f"Number of PCs that explain more than 1% of variance - early: {early_num_pcs_above_1_percent}")

print(f"Number of PCs needed to explain 90% of variance - late: {late_num_pcs_90}")
print(f"Number of PCs that explain more than 1% of variance - late: {late_num_pcs_above_1_percent}")

The Efficiency in the early rotation using Pseudo-Determinant : 3811405967.637319
The Efficiency in the late rotation using Pseudo-Determinant : 2932852988.510496
Number of PCs needed to explain 90% of variance - early: 4
Number of PCs that explain more than 1% of variance - early: 7
Number of PCs needed to explain 90% of variance - late: 3
Number of PCs that explain more than 1% of variance - late: 7


In [262]:
import csv

# Create a dictionary with the parameters
data = {
    'early_det_value': early_det_value,
    'late_det_value': late_det_value,
    'pseudo_early_cov_matrix': pseudo_early_cov_matrix,
    'pseudo_late_cov_matrix': pseudo_late_cov_matrix,
    'pseudo_efficiency': pseudo_efficiency,
    'pseudo_efficiency_late': pseudo_efficiency_late,
    'early_num_pcs_90': early_num_pcs_90,
    'early_num_pcs_above_1_percent': early_num_pcs_above_1_percent,
    'late_num_pcs_90': late_num_pcs_90,
    'late_num_pcs_above_1_percent': late_num_pcs_above_1_percent
}

# Define the filename
filename = 'Subject01_rotation_analysis.csv'

# Write the data to the CSV file
with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(data.keys())  # Write header
    writer.writerow(data.values())  # Write values

print(f"Data saved to {filename}")


Data saved to Subject01_rotation_analysis.csv


In [263]:
# Define the path to the CSV file to store the early data
Subejct01_alljoints_path = os.path.join('C:/Users/86153/Desktop/MCSRsystem/SetupV2/exampleplots/', "Subejct01_all_joints_df.csv")
# Save the all_fish_df to the CSV file
all_joints_df.to_csv(Subejct01_alljoints_path, index=False)

In [264]:
rotation_angles_df = pd.DataFrame(rotation_angles)
rotation_angles_path = os.path.join('C:/Users/86153/Desktop/MCSRsystem/SetupV2/exampleplots/', "Subejct01_angerror_df.csv")
rotation_angles_df.to_csv(rotation_angles_path,index = False)

# Test for the num_components

In [265]:


# opls = OPLS(10)
# self = opls.fit(early_data,early_angerror)
# TP = self.T_ortho_ @ np.transpose(self.P_ortho_)
# early_reduandant_cov_matrix = np.cov(TP)
# early_reduandant_det_value = np.linalg.det(early_reduandant_cov_matrix)
# Z = opls.transform(early_data)
# early_related_cov_matrix = np.cov(Z)
# early_related_det_value = np.linalg.det(early_related_cov_matrix)
# pseudo_det = effective_pseudo_determinant(early_reduandant_cov_matrix, threshold=1e-1)
# pseudo_det_related = effective_pseudo_determinant(early_related_cov_matrix, threshold=1e-1)
# pseudo_efficiency  = pseudo_det/pseudo_det_related
# pseudo_early_cov_matrix = effective_pseudo_determinant(early_cov_matrix, threshold=1e-1)



# opls = OPLS(10)
# self = opls.fit(late_data,late_angerror)
# TP = self.T_ortho_ @ np.transpose(self.P_ortho_)
# late_reduandant_cov_matrix = np.cov(TP)
# late_reduandant_det_value = np.linalg.det(early_reduandant_cov_matrix)
# Z = opls.transform(late_data)
# late_related_cov_matrix = np.cov(Z)
# late_related_det_value = np.linalg.det(late_related_cov_matrix)
# pseudo_det_late = effective_pseudo_determinant(late_reduandant_cov_matrix, threshold=1e-1)
# pseudo_efficiency_late  = pseudo_det_late/pseudo_det_related_late
# pseudo_det_related_late = effective_pseudo_determinant(late_related_cov_matrix, threshold=1e-1)
# pseudo_late_cov_matrix = effective_pseudo_determinant(late_cov_matrix, threshold=1e-1)

# print(f"The genearal variance in the early rotation using Pseudo-Determinant: {pseudo_early_cov_matrix}")
# print(f"The genearal variance in the late rotation using Pseudo-Determinant : {pseudo_late_cov_matrix}")

# print(f"The Efficiency in the early rotation using Pseudo-Determinant : {pseudo_efficiency}")
# print(f"The Efficiency in the late rotation using Pseudo-Determinant : {pseudo_efficiency_late}")

In [266]:
# Define the range of input values for OPLS
opls_inputs = range(5, 41)

# Empty lists to store the results for each opls input
early_pseudo_det_values = []
late_pseudo_det_values = []
early_efficiency_values = []
late_efficiency_values = []

# Loop through each opls input value
for n_components in opls_inputs:
    # Initialize and fit OPLS model for early data
    opls_early = OPLS(n_components)
    self_early = opls_early.fit(early_data, early_angerror)
    TP_early = self_early.T_ortho_ @ np.transpose(self_early.P_ortho_)
    early_redundant_cov_matrix = np.cov(TP_early)
    
    Z_early = opls_early.transform(early_data)
    early_related_cov_matrix = np.cov(Z_early)
    
    # Compute pseudo-determinants and efficiency for early data
    pseudo_det_early_redundant = effective_pseudo_determinant(early_redundant_cov_matrix, threshold=1e-1)
    pseudo_det_early_related = effective_pseudo_determinant(early_related_cov_matrix, threshold=1e-1)
    pseudo_efficiency_early = pseudo_det_early_redundant / pseudo_det_early_related
    pseudo_early_cov_matrix = effective_pseudo_determinant(early_cov_matrix, threshold=1e-1)
    
    # Store the results for early data
    early_efficiency_values.append(pseudo_efficiency_early)

    # Initialize and fit OPLS model for late data
    opls_late = OPLS(n_components)
    self_late = opls_late.fit(late_data, late_angerror)
    TP_late = self_late.T_ortho_ @ np.transpose(self_late.P_ortho_)
    late_redundant_cov_matrix = np.cov(TP_late)
    
    Z_late = opls_late.transform(late_data)
    late_related_cov_matrix = np.cov(Z_late)
    
    # Compute pseudo-determinants and efficiency for late data
    pseudo_det_late_redundant = effective_pseudo_determinant(late_redundant_cov_matrix, threshold=1e-1)
    pseudo_det_late_related = effective_pseudo_determinant(late_related_cov_matrix, threshold=1e-1)
    pseudo_efficiency_late = pseudo_det_late_redundant / pseudo_det_late_related
    pseudo_late_cov_matrix = effective_pseudo_determinant(late_cov_matrix, threshold=1e-1)
    
    # Store the results for late data
    late_efficiency_values.append(pseudo_efficiency_late)




In [267]:
# Print the results in a table format
print("OPLS Components   | Early Efficiency   | Late Efficiency")
for i, n in enumerate(opls_inputs):
    print(f"{n:<16} | {early_efficiency_values[i]:<16} | {late_efficiency_values[i]}")

OPLS Components   | Early Efficiency   | Late Efficiency
5                | 374.7139682624249 | 823.5118676570173
6                | 4324.589579323188 | 9595.29192680838
7                | 41979.10979960389 | 424117.6842133787
8                | 193603.65698887553 | 1456305.7264347267
9                | 293185.1921620506 | 9741910.63631922
10               | 13791769.208351051 | 32846239.954426482
11               | 35022511.765134715 | 69078274.28146312
12               | 65989725.786085166 | 487012128.9884738
13               | 152795503.24758956 | 799611769.7974821
14               | 238601929.43347704 | 729758373.0826308
15               | 520356107.24929893 | 637717810.0752219
16               | 740566457.2032137 | 1168679406.2643082
17               | 443754030.6221066 | 7726334955.258456
18               | 387380774.9212312 | 3723031378.520125
19               | 391348196.77161545 | 2954917530.40068
20               | 3811405967.637319 | 2932852988.510496
21               | 2873

In [269]:
# Plot the efficiencies
plt.figure(figsize=(10, 6))
plt.plot(opls_inputs, early_efficiency_values, label="Early Efficiency", marker='o')
plt.plot(opls_inputs, late_efficiency_values, label="Late Efficiency", marker='s')

# Labeling the plot
plt.xlabel("OPLS Components")
plt.ylabel("Efficiency")
plt.title("Efficiency vs. OPLS Components")
plt.legend()
plt.grid(True)

# Show the plot
plt.show()